In [161]:
import warnings 
warnings.filterwarnings("ignore")
from sklearn.linear_model import ElasticNet

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder , PolynomialFeatures, Normalizer ,normalize
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split ,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor
from scipy.stats import skew
train = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/train.csv')
test = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/test.csv')
sample_submission = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission.csv')

In [162]:

data = pd.concat([train , test] , axis= 0)
test_id = test["Id"]
data.drop(["Id"] ,axis=1, inplace=True)
data["Age"] = data["YrSold"] - data["YearBuilt"]
data["Agere"] = data["YrSold"] - data["YearRemodAdd"]
data["yaxsi"] = data["1stFlrSF"]+data["2ndFlrSF"]-data["LowQualFinSF"]
data["umumi"] = data["1stFlrSF"]+data["2ndFlrSF"]
data["LotFrontage"] = data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
data['MSSubClass'] = data['MSSubClass'].apply(str)
data['MoSold'] = data['MoSold'].astype(str)



In [163]:
str_cols = data.select_dtypes("O").columns
num_cols = data.select_dtypes("number").columns
num_cols = list(num_cols)
num_cols.append("Agere")
no_cols = ["MiscFeature" , "Fence" , "GarageCond" , "GarageQual" , "GarageFinish" , "FireplaceQu" , "BsmtFinType2" , "BsmtExposure", "BsmtCond" , "BsmtQual" , "Alley"]
for col in data.columns:
    if col != "SalePrice":
        if col in no_cols:
            data[col]=data[col].fillna("None")
        elif col in str_cols:
            data[col]=data[col].fillna(data[col].mode()[0])
        elif col in num_cols:
            data[col]=data[col].fillna(0)
np.array(no_cols)
data["TotalHouse"] = data["TotalBsmtSF"] + data["1stFlrSF"] + data["2ndFlrSF"]   
data["qualtot"] = data["TotalHouse"] * data["OverallQual"]


In [164]:
def detect_outliers(df,col):
    Q1 = df[col].quantile(0.005)
    Q3 = df[col].quantile(0.995)
    IQR = Q3-Q1
    lwr_bnd = Q1 - IQR*1.5
    hgh_bnd= Q3 + IQR*1.5
    df.drop(df[df[col]>hgh_bnd].index ,axis =0, inplace=True)
    df.drop(df[df[col]<lwr_bnd].index , axis = 0, inplace=True)


In [165]:
ohe  = OneHotEncoder(drop = "first")
ohe.fit(data[str_cols])
ohe_data=ohe.transform(data[str_cols]).toarray()
ohe_df = pd.DataFrame(ohe_data , columns = ohe.get_feature_names_out())

In [166]:
if list(str_cols) in list(data.columns):
    data.drop(str_cols , inplace=True ,axis = 1)
data.index=range(len(data))

ohe_df.index=range(len(ohe_df))

In [167]:
data=pd.concat([data, ohe_df], axis=1)

In [168]:
# ct = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), a.select_dtypes(include=['number']).columns)], remainder='passthrough')
# ct.fit_transform(data)


In [169]:
# cte = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), pd.Series((list(data.select_dtypes(include=['number']).columns).remove("SalePrice"))).astype("O"))], remainder='passthrough')
# cte.fit_transform(data)
# data.select_dtypes("number")

skewed_feats = data[num_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness = skewness[abs(skewness) > 0.75]
skewness.head()
from scipy.special import boxcox1p
skewed_features = skewness.index
for col in skewed_features:
    data[col] = normalize([np.array(data[col])])
skewed_feats = abs(data[num_cols].apply(lambda x: skew(x.dropna()))).sort_values(ascending=False)
abs(skewed_feats).head(30)



In [170]:

train=data[data['SalePrice'].notnull()]
for col in list(num_cols):
    detect_outliers(train,col)
test=data[data['SalePrice'].isnull()]
test.drop(['SalePrice'] , inplace=True , axis = 1)

X=train.drop('SalePrice', axis=1)
Y=train['SalePrice']


In [171]:
ct = ColumnTransformer([('encoder', OneHotEncoder(handle_unknown='ignore'), X.select_dtypes(include=['object']).columns)], remainder='passthrough')
X = ct.fit_transform(X)

X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size=0.3 , random_state=42)
test= ct.transform(test)


In [172]:
def Poly_F(x , y , i , test ):
    pf = PolynomialFeatures(degree=i)
    pf.fit(x)
    x_poly = pd.DataFrame(pf.transform(x) , columns = pf.get_feature_names_out())
    pfe = PolynomialFeatures(degree=i)
    pfe.fit(test)
    test = pd.DataFrame(pfe.transform(test) , columns = pfe.get_feature_names_out())

    X_train , X_test , Y_train , Y_test = train_test_split(x_poly , y , test_size=0.3 , random_state=42)
    lr = LinearRegression()
    lr.fit(X_train , Y_train)
    # rmse_t = np.sqrt(mean_squared_error(Y_train , lr.predict(X_train)))
    # rmse= np.sqrt(mean_squared_error(Y_test , lr.predict(X_test)))
    return lr.predict(test)

In [173]:
# def enet(x , y , i , j):
#     enet = ElasticNet(alpha= i  , l1_ratio=j)
#     X_train , X_test , Y_train , Y_test = train_test_split(x , y , test_size=0.25 , random_state=42)
#     enet.fit(X_train , Y_train)
#     rmse_t = np.sqrt(mean_squared_error(Y_train , enet.predict(X_train)))
#     rmse= np.sqrt(mean_squared_error(Y_test , enet.predict(X_test)))
#     return rmse_t , rmse , i , j

In [174]:
# best_enet = {}
# best_enet = dict(best_enet)
# best_enety = {}
# best_enety = dict(best_enety)
# for i in [5,20,80]:
#     for j in [5,20,80]:
#         a , b , c , d= enet(X ,Y ,i/100 , j/100)
#         best_enet[f" i {c} , j {d}"] = b
#         best_enety[f" i {c} , j {d}"] = a


In [175]:
# print(print(min(best_enet, key=best_enet.get)))
# print(print(min(best_enety, key=best_enety.get)))
# print(best_enet )

In [176]:
#KNN - 0.39
#knn = KNeighborsClassifier()
#knn.fit(X_train , Y_train)
for col in data:
    if data[col].isnull().sum()/data[col].count()>0:
        print(col ,data[col].isnull().sum()/data[col].count() )

SalePrice 0.9993150684931507


In [177]:
#Poly degree 2
lr = LinearRegression()
lr.fit(X , Y)


LinearRegression()

In [178]:
# Skewed data nedi oyrenirem

In [179]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split

def rmsle_cv(model):
    kf = KFold(5, shuffle=True, random_state=42).get_n_splits(X)
    rmse= np.sqrt(-cross_val_score(model, X, Y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [192]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler , StandardScaler

lasso = make_pipeline(Normalizer(), Lasso(alpha =0.0005, random_state=1))
LassoMd = lasso.fit(X,Y)
score = rmsle_cv(lasso)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Kernel Ridge score: 37794.4099 (6672.3346)



In [200]:
enet = ElasticNet(alpha=0.4,l1_ratio=0.9,max_iter=10000 )
enet.fit(X , Y)

26270.7093

26270.7093

In [202]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.4, l1_ratio=.9, max_iter=10000 ))
enet = ENet.fit(X,Y)
score = rmsle_cv(ENet)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
28086.0158

Kernel Ridge score: 25851.9697 (5027.7773)



28086.0158

In [203]:


filename = "/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission2.csv"
import os
test_pred =enet.predict(test)
sample_submission['SalePrice'] = test_pred
if os.path.exists(filename):
    os.remove(filename)
sample_submission.to_csv(filename, index=False) 
sample_submission.head()


,Id,SalePrice
0,1461,116559.922222
1,1462,196536.925679
2,1463,182036.972616
3,1464,197219.156298
4,1465,198388.260881
